<a href="https://colab.research.google.com/github/alisabak/LINdoscope2023/blob/main/notebooks/DLC_Colab_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab for standard (single animal) projects!
This notebook is a modified copy from [Github](https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb), originally written by Mackenzie Mathis; publication: Mathis et al 2018, https://doi.org/10.1038/s41593-018-0209-y.

It has been edited for the 2023 EMBO practical course LINdoscope: https://www.lindoscope.com.

https://github.com/DeepLabCut/DeepLabCut

![alt text](https://camo.githubusercontent.com/71523c879284afa1c51b8489491f06d24d1ff02d75dca9f71078535ee6b1978e/68747470733a2f2f696d616765732e73717561726573706163652d63646e2e636f6d2f636f6e74656e742f76312f3537663664353163396637343536366635356563663237312f313632383235303030343232392d4b565944374a4a5648594546444a33324c39564a2f444c436c6f676f323032312e6a70673f666f726d61743d3130303077)

This notebook illustrates how to use colab to:
- load a pretrained model
- analyze novel videos, filter data, create videos

This notebook demonstrates the necessary steps to use DeepLabCut for your own project. This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper.

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

**(Be sure to click "RESTART RUNTIME" if it is displayed above above before moving on !)**

## Link your Google Drive

1. First, add a shortcut to a pretrained DLC model into you google drive. To do so, right click on the desired folder -> Organise -> Add shortcut -> My Drive -> Add

2. Next, add [a shortcut to] a folder with videos that you want to analyse to your Google Drive

3. Now, let's link to your GoogleDrive. Run the cell below and follow the authorization instructions



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup your project variables
Modify variables **PrertrainedModelFolderName** and **VideoFolderName** to names of a folder with a pretrained model and to the videos you would like to analyse, respectively.


In [ ]:
# PLEASE EDIT THESE:
PrertrainedModelFolderName = 'For LINdoscope test/top_transparent-Alisa-2023-02-08'
VideoFolderName = 'For LINdoscope test/videos'
VideoType = 'mp4'

#don't edit these:
videofile_path = ['/content/drive/My Drive/'+VideoFolderName] #Enter the list of videos or folder to analyze.
videofile_path

#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/drive/My Drive/'+PrertrainedModelFolderName+'/config.yaml'
path_config_file

In [ ]:
import deeplabcut
deeplabcut.__version__

## Start Analyzing videos
This function analyzes your videos.

The results are stored in hd5 file in the same directory where the video resides. The data is also exported in comma-separated values format (.csv).

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype=VideoType, shuffle=1, save_as_csv=True)

You can also now filter your data to smooth any small jitters:

In [ ]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype=VideoType, shuffle=1)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
import matplotlib
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=1, filtered=True)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType, shuffle=1, filtered=True)